## CRISM Image Preprocessing
	1. Download data and preprocess in CAT ENVI: Convert PDS to CAT (rotate data 180 degrees). 
    2. Layer "l" and "s" image files using record_layered_data function (which also gets rid of NULL values); and save as Pickle.
	3.  Reduce wavelengths to match CRISM & USGS. 
    4. Recalculate k-values (if wavelengths have changed from previous)


In [ ]:
from preprocessing.CRISM_preprocess_helpers import *
from utils.plotting import *
from utils.constants import *
import pandas as pd

from numpy import genfromtxt


1. Download from Data Explorer, import into CAT ENVI, rotate 180 degrees, save as standard CAT ENVI, save the z profiles of a pixel on each image. Layer l and s using record_layered_data.


2. Save total set of wavelengths as single .pickle (from s and l pixel wavelengths)


In [ ]:
# CRISM_DIR = PREPROCESSED_DATA + "CRISM/frt00010628_07/"
CRISM_DIR = PREPROCESSED_DATA + "CRISM/frs0002edd6_01/"

In [ ]:
lW = genfromtxt(CRISM_DIR + 'lpixel.csv', delimiter=',', skip_header=1)
sW = genfromtxt(CRISM_DIR + 'spixel.csv', delimiter=',', skip_header=1)

# Combine short and long
W = np.concatenate((sW, lW))

with open(CRISM_DIR + "wavelengths.pickle", 'wb') as F:
    pickle.dump(W, F)

3. Reduced wavelengths: RW_CRISM and RW_USGS.
Meaning: CRISM data has a certain wavelengths, and USGS has different ones. So, we need to find the mutually compatible set. This is what reducing wavelengths is.


In [ ]:
CRISM_reduced, USGS_reduced = record_CRISM_USGS_reduced_wavelengths(CRISM_img_dir = "frt00010628_07")


4. Recalculate k-values
Usually we run this on the server/HPC because it takes a while (and can use many threads to infer k in different wavelengths at the same time).

In [ ]:
from preprocessing.estimatek import estimate_all_USGS_k
# True means to match CRISM wavelengths
estimate_all_USGS_k(True)

5. [OPTIONAL] Save PDF of image

In [ ]:
CRISM_DIR = PREPROCESSED_DATA + "CRISM/frt00010628_07/"

CUR_IMG_NAME = "frt00010628_07_if165_LAYERED.pickle"
with open(CRISM_DIR + CUR_IMG_NAME, 'rb') as handle:
    img = pickle.load(handle)
plot_CRISM_img(img, CRISM_DIR+"frt00010628.pdf")


6. [OPTIONAL] Identify subsection of image to use for testing
Save as Pickle and as PDF for viewing.

In [ ]:
CUR_IMG_NAME = "frt00010628_07_if165_LAYERED.pickle"
with open(CRISM_DIR + CUR_IMG_NAME, 'rb') as handle:
    loaded_img = pickle.load(handle)
view = imshow(loaded_img, bands= (200, 71, 18), title="Layered frt00010628 I/F")

In [ ]:

# rows
rowstart = 40
rowend = 60

#  cols
colstart = 80
colend = 100

coords = [ colstart, colend, rowstart, rowend]

# For plotting below, X corresponds to columns

plot_zoomed_sectioned_CRISM(loaded_img, coords, CRISM_DIR+"Section-frt00010628.pdf")

SUBIMG = loaded_img[rowstart:rowend,colstart:colend, :]
view = imshow(SUBIMG, 
              bands= (100, 71, 18), 
              title="Section frt00010628 I/F")

In [ ]:
# Save called-out section.
with open(CRISM_DIR + "frt00010628_subsection.pickle", 'wb') as F:
    pickle.dump(SUBIMG, F)

In [ ]:
plot_CRISM_img(SUBIMG, CRISM_DIR+"frt00010628_subsection.pdf")

## Testing get_CRISM_data make sure it reduces wavelengths correctly

In [ ]:
from utils.access_data import *
CRISM_DIR = PREPROCESSED_DATA + "CRISM/frt00010628_07/"

img = get_CRISM_data(file_name=CRISM_DIR+"frt00010628_07_if165_LAYERED.pickle", 
               img_dir=CRISM_DIR)
print("Image shape " + str(img.shape))

In [ ]:
w=get_CRISM_wavelengths("frt00010628_07", CRISM_match=False)
len(w)

In [ ]:

file_name=CRISM_DIR+"frt00010628.pickle"
img_dir=CRISM_DIR
with open(file_name, 'rb') as handle:
    loaded_img = pickle.load(handle)



In [ ]:
loaded_img.shape

In [ ]:
full_w = get_CRISM_wavelengths("frt00010628_07", CRISM_match=False)
len(full_w)

In [ ]:
frsDIR =  PREPROCESSED_DATA + "CRISM/frs0002edd6_01/"
file_name=frsDIR+"frs0002edd6_LAYERED.pickle"
img_dir=frsDIR
with open(file_name, 'rb') as handle:
    loaded_img = pickle.load(handle)
print(loaded_img.shape)
full_w = get_CRISM_wavelengths("frs0002edd6_01", CRISM_match=False)
print(len(full_w))

In [ ]:
orig_wavelengths = get_CRISM_wavelengths(img_dir, True)
matched_wavelengths = get_CRISM_RWs()

indices = np.argwhere(np.isin(orig_wavelengths, matched_wavelengths)).flatten()
return np.take(loaded_img, indices, axis=2)